# `swift_too` module

## Swift_Data example - querying Swift SDC data and downloading it

### API version = 1.2, `swifttools` = 2.4.6

#### Author: Jamie A. Kennea (Penn State)

In [1]:
from swifttools.swift_too import Data
from swifttools.swift_too import ObsQuery

### Introduction

We've demonstrated that with classes such as Swift_ObsQuery we can find out what Swift has observed, but what if we want to download the data associated with an observation? Before you had to go to a web interface to download the data you wanted. More confusingly, the data usually reside in one of two places: the archive or the quicklook area. Typically recent observations exist in quicklook, and older observations are in the archive. 


### Finding an interesting observation

`Swift_Data` aims to make fetching observation data automated, and somewhat abstracted, so you don't have to care about if data are from quicklook or not. Let's find a recent observation, and download it. How about the first few observations of Jan 10th, 2021? As of `swifttools` 2.3, you can shorten the names of the classes by removing the `Swift_` so we'll use just `Data` and for the next example, `ObsQuery`.

In [2]:
oq = ObsQuery(begin="2022-01-10", length=0.1)
oq

2022-01-09 23:52:02,2022-01-10 00:12:58,NGC5986,00045907009,1115,141
2022-01-10 00:13:02,2022-01-10 00:20:57,PSZ1_G357.43+30.60,00085651001,410,65
2022-01-10 00:21:02,2022-01-10 00:51:01,GRB 220101A,01091527009,1610,189
2022-01-10 00:51:02,2022-01-10 01:12:01,OGP_4313,03103018001,1080,179
2022-01-10 01:12:02,2022-01-10 01:28:57,MRK 817,00096450022,835,180
2022-01-10 01:29:02,2022-01-10 01:36:01,NGC 5398,03110928017,275,144
2022-01-10 01:36:02,2022-01-10 01:57:59,HEN 3-1103,00083966007,1260,57
2022-01-10 01:58:02,2022-01-10 02:23:00,RX J0134.2-4258,00036528060,1355,143
2022-01-10 02:23:02,2022-01-10 02:44:58,OGP_3952,03103167003,1185,131


OK, so that observation fo NGC 5986 is interesting, so let's check it out. The important identifier for any observation is the Observation Number (or Observation ID). In the table above, this is given in SDC format, which is a importantly, a string (those leading zeros are required). So for this observation, the obsid is `00045907009`. However, we don't have to make transcription errors here, it's the first in the oq data structure, i.e...

In [3]:
oq[0]

2022-01-09 23:52:02,2022-01-10 00:12:58,NGC5986,00045907009,1115,141


### Demonstrating the `Swift_Data` class

So let's demonstrate use of the Swift_Data class. As usual, it can be set up by creating the class and then setting the parameters, then submitting using the `submit` method, or you can pass the parameters as arguments to the class. We'll do the former first.

In [4]:
data = Data()
data.obsid = oq[0].obsid

OK so this sets up that we want to fetch data associated with the obsid for NGC5986. Next we have a few options to set. Firstly, there are 5 types of data that can be downloaded. These are:

- `auxil` : Auxillary data required to process Swift data, including Two Line Element (TLE) to calculated an ephemeris, and spacecraft housekeeping.
- `log` : Log files from the SDC processing. Of interest, but usually inessential for data analysis.
- `xrt` : Data for Swift's X-ray Telescope.
- `bat` : Data for Swift's Burst Alert Telescope 
- `uvot` : Data for Swift's UV/Optical Telescope

In order to select which data you wish to download, simply set the parameter listed above to true, so if you want XRT data, set `xrt = True`. If you just want to get everything, you can set `all = True` as a shortcut. Let's get XRT, BAT and UVOT data for this observation, forget about the logs. FYI `auxil` data come down by default, as you always need those if you're going to run processing pipeline tools like `xrtpipeline`.

In [5]:
data.xrt = True
data.uvot = True
data.bat = True

So in terms of basic configuration, this is pretty much it. Next step is to query the existance of data. This is done by submitting the job. 

In [6]:
data.submit()

True

OK that appeared to have worked, so let's look at the files available. In a Jupyter Notebook, just type in the name of the `Swift_Data` object we created.

In [7]:
data

00045907009/auxil,SWIFT_TLE_ARCHIVE.txt.22018.37267907.gz,Auxillary file
'',sw00045907009pat.fits.gz,Auxillary attitude file
'',sw00045907009pjb.par.gz,Auxillary file
'',sw00045907009pob.cat.gz,Auxillary catalog
'',sw00045907009ppr.par.gz,Auxillary file
'',sw00045907009s.mkf.gz,Auxillary file
'',sw00045907009sao.fits.gz,Auxillary file
'',sw00045907009sat.fits.gz,Auxillary attitude file
'',sw00045907009sen.hk.gz,Auxillary housekeeping
'',sw00045907009sti.fits.gz,Auxillary file
'',sw00045907009uat.fits.gz,Auxillary attitude file


OK so what you should see above is a list of all the files, including the subdirectories they will exist in, along with basic description of each file. Importantly, at this stage, the data are not yet downloaded. Let's look in depth at one of these files.

In [8]:
data[0]

filename,SWIFT_TLE_ARCHIVE.txt.22018.37267907.gz
path,00045907009/auxil
url,https://heasarc.gsfc.nasa.gov/FTP/swift/data/obs/2022_01/00045907009/auxil/SWIFT_TLE_ARCHIVE.txt.22018.37267907.gz
quicklook,False
type,Auxillary file


OK so a few things here. Firstly, from the `url` you can see where the data reside online. As this is a nasa.gov address, this is the US data center. If you wanted to retrieve from the UK data center, simply pass the parameter `uksdc = True`. Similarly if you wish to use the Italian SDC, use `itsdc = True`.


Secondly, this is quicklook data. By default `Swift_Data` looks in the archive first, but if the data is recent, it will look in quicklook. Note that quicklook data are by their nature preliminary, and can evolve with time. For US and Italian SDC you'll see that the URL has a version number ('009') included, this number increments when the data are changed. For UK SDC, this does not happen, however, you can just redownload data to get the most up-to-date value. 

For published papers, we recommend using data from the archive if available.

### Downloading the data

Final step is to download the data to disk, so we can process the data using our HEAsoft tools. This is perfomed utilizing the `download()` method. If we want to specify the directory to download, simply use the `outdir` parameter, either as a class parameter, or an argument to download. Let's put in in our computer's Downloads directory.

In [9]:
data.download(outdir="~/Downloads")

True

If you have `tqdm` installed, you should have seen a progress bar showing how many files were downloaded. If not, then you'll just see a simple text message. Now the data should be in the Downloads directory, in a directory named after the `obsid` of the data we requested. Note that the files inside are gzipped by default, but do not need to be decompressed to be used with the HEAsoft tools, so you can save some disk space!

### Download data in one line

Last thing we'll demonstrate is downloading data in one line of code. This is very simple, just pass the arguments when you intiate the class, and it will download the data itself. If you want to do this, and not download the data automatically, pass the argument `fetch=False`. We'll fetch the 3rd observation above, but just the XRT data:

In [10]:
newdata = Data(obsid=oq[2].obsid, xrt=True, outdir="~/Downloads")

That should have downloaded the `auxil` and `xrt` data for that observation in into your Downloads directory. Let's look at the file structure.

In [11]:
newdata

01091527009/auxil,SWIFT_TLE_ARCHIVE.txt.22020.02942007.gz,Auxillary file
'',sw01091527009pat.fits.gz,Auxillary attitude file
'',sw01091527009pjb.par.gz,Auxillary file
'',sw01091527009pob.cat.gz,Auxillary catalog
'',sw01091527009ppr.par.gz,Auxillary file
'',sw01091527009s.mkf.gz,Auxillary file
'',sw01091527009sao.fits.gz,Auxillary file
'',sw01091527009sat.fits.gz,Auxillary attitude file
'',sw01091527009sen.hk.gz,Auxillary housekeeping
'',sw01091527009sti.fits.gz,Auxillary file
'',sw01091527009uat.fits.gz,Auxillary attitude file


### Updating data, fetching additional instruments

OK, so what if you want to update directory, say the data has been updated. Well we would recommend that you delete the old data, and redownload, and to encourage this, if you try to download again, you'll get an errors for all the files that already exist:

In [12]:
newdata = Data(obsid=oq[2].obsid, xrt=True, outdir="~/Downloads")

ERROR: SWIFT_TLE_ARCHIVE.txt.22020.02942007.gz exists (set clobber=True to override this).
ERROR: sw01091527009pat.fits.gz exists (set clobber=True to override this).
ERROR: sw01091527009pjb.par.gz exists (set clobber=True to override this).
ERROR: sw01091527009pob.cat.gz exists (set clobber=True to override this).
ERROR: sw01091527009ppr.par.gz exists (set clobber=True to override this).
ERROR: sw01091527009s.mkf.gz exists (set clobber=True to override this).
ERROR: sw01091527009sao.fits.gz exists (set clobber=True to override this).
ERROR: sw01091527009sat.fits.gz exists (set clobber=True to override this).
ERROR: sw01091527009sen.hk.gz exists (set clobber=True to override this).
ERROR: sw01091527009sti.fits.gz exists (set clobber=True to override this).
ERROR: sw01091527009uat.fits.gz exists (set clobber=True to override this).
ERROR: sw01091527009x.mkf.gz exists (set clobber=True to override this).
ERROR: sw01091527009xpcw3po_cl.evt.gz exists (set clobber=True to override this).
ER

However, we can overwrite the existing data, by (as the error message suggests), setting `clobber = True`.

In [13]:
newdata.clobber = True
newdata.download()

True

As of `swifttools 2.4.6` the behavior of `clobber` changed to work on a file by file basis. So now you can fetch data for an additional instrument, without setting `clobber=True`. Remember however, `auxil=True` by default, so we'll have to set it to `False` so it doesn't attempt to redownload that data.


In [14]:
newdata = Data(
    obsid=oq[2].obsid, uvot=True, auxil=False, outdir="~/Downloads"
)

So the above listing will show the UVOT data files that you just downloaded to add to your XRT and Auxillary data. 

### Fetching specific file types

If you want to only fetch specific file types, as of `swifttools 2.4.6` you can do this with the optional `match` parameter. You use this to supply unix style filename pattern matches, and `Data` will fetch only files that match these. If you give multiple patterns, files matching any of the given patterns will be downloaded. So for example, if we wish to download just the BAT event data for a given GUANO event:

In [15]:
from swifttools.swift_too import GUANO

g = GUANO(limit=10)
g

GBM GRB,2022-06-24 07:34:56,50,200 (194),00083413002
FRB,2022-06-24 05:26:27.568224,0,200,03103630001
GBM GRB,2022-06-24 02:58:35.160001,50,200,00015234002
CALET_GRB,2022-06-24 02:58:32.530000,0,90 (200),00015234002
FRB,2022-06-24 00:15:05.912605,0,200 (186),01111672002
GBM GRB,2022-06-23 15:33:56.430000,50,200 (128)*,03112057001
INTEGRAL_GRB,2022-06-23 15:33:54.740000,0,90 (128)*,03112057001
INTEGRAL_GRB,2022-06-23 04:13:38.960000,0,90,00031540063
GBM GRB,2022-06-22 13:28:33.860000,50,200,00074842079
FRB,2022-06-22 11:21:04.460308,0,90,00010905161


In [16]:
Data(obsid=g[0].obsid,
     bat=True,
     match='*.evt*',
     fetch=False)

00083413002/bat/event,sw00083413002bevshpo_uf.evt.gz,BAT pointed unfiltered event file


So you can see here that it has only listed the BAT event file, as we specified `bat=True` and matched to `*.evt.*`. However, we probably need the contents of the `auxil` directory too, as we didn't specify a match for those, so let's add another match for that. We can pass as many matches as we like in an array.

In [17]:
Data(obsid=g[0].obsid,
     bat=True,
     match=['*.evt*','*/auxil/*'],
     fetch=False)

00083413002/auxil,SWIFT_TLE_ARCHIVE.txt.22174.58920454.gz,Auxillary file
'',sw00083413002pat.fits.gz,Auxillary attitude file
'',sw00083413002pjb.par.gz,Auxillary file
'',sw00083413002pob.cat.gz,Auxillary catalog
'',sw00083413002ppr.par.gz,Auxillary file
'',sw00083413002s.mkf.gz,Auxillary file
'',sw00083413002sao.fits.gz,Auxillary file
'',sw00083413002sat.fits.gz,Auxillary attitude file
'',sw00083413002sen.hk.gz,Auxillary housekeeping
'',sw00083413002sti.fits.gz,Auxillary file
'',sw00083413002uat.fits.gz,Auxillary attitude file


So now we returned all the files in the `auxil` directory and the BAT event file.

That's pretty much all there is to it! Enjoy your data. 